In [1]:
from DTG.classes.LLM import *
from DTG.prompts import prompt_funcs
import json, time

In [2]:
INPUT_INFO = 'NPC personality : lazy and shy . NPC mood : confused and amused. Convesation features : dancing, great move, tripping'

In [3]:
def get_dialogue_json(input_info):
    t_0 = time.time()
    starting_time = time.time()
    performance_dict = {
        'tags' : 0,
        'abstract' : 0,
        'endings' : 0,
        'tree' : 0,
        'formatted_json' : 0
    }
    llm = LLM()
    print(f'Generating tags.')
    tags_by_llm = llm.send_message_for_code(prompt_funcs.tag_prompt(input_info), model='gpt-4o-mini')
    print(f'Tags generated in {time.time() - starting_time} seconds.')
    performance_dict['tags'] = time.time() - starting_time
    starting_time = time.time()
    print(f'Generating abstract.')
    abstract = llm.send_message_for_code(prompt_funcs.abstract_prompt(tags_by_llm, input_info))
    print(f'Abstract generated in {time.time() - starting_time} seconds.')
    performance_dict['abstract'] = time.time() - starting_time
    starting_time = time.time()
    print(f'Generating possible endings.')
    endings_by_llm = llm.send_message_for_code(prompt_funcs.possible_ending_prompt(abstract), model='o1-mini')
    print(f'Possible endings generated in {time.time() - starting_time} seconds.')
    performance_dict['endings'] = time.time() - starting_time
    starting_time = time.time()
    print(f'Generating dialogue tree.')
    tree_by_llm = llm.send_message_for_code(prompt_funcs.dialogue_prompt(abstract, endings_by_llm, tags_by_llm), model='o1-preview')
    print(f'Dialogue tree generated in {time.time() - starting_time} seconds.')
    performance_dict['tree'] = time.time() - starting_time
    starting_time = time.time()
    print(f'Formatting JSON.')
    formatted_tree = llm.send_message(prompt_funcs.format_prompt(tree_by_llm), model='gpt-4o-mini')
    print(f'JSON formatted in {time.time() - starting_time} seconds.')
    performance_dict['formatted_json'] = time.time() - starting_time
    starting_time = time.time()
    print(f'Total time taken : {time.time() - t_0} seconds.')
    print(f'Performance dict : {performance_dict}')
    formatted_tree_json = json.loads(formatted_tree)
    #formatted_endings_json = json.loads(endings_by_llm)
    return formatted_tree_json, endings_by_llm


In [4]:
#tree, endings = get_dialogue_json(INPUT_INFO)

In [5]:
from DTG.classes.OutputClasses import Tree, PossibleEndings

In [6]:
def get_dialogue_json(input_info):
    t_0 = time.time()
    starting_time = time.time()
    performance_dict = {
        'tags' : 0,
        'abstract' : 0,
        'endings' : 0,
        'tree' : 0
    }
    llm = LLM()
    print(f'Generating tags.')
    tags_by_llm = llm.send_message_for_code(prompt_funcs.tag_prompt(input_info), model='gpt-4o-mini')
    print(f'Tags generated in {time.time() - starting_time} seconds.')
    performance_dict['tags'] = time.time() - starting_time
    starting_time = time.time()
    print(f'Generating abstract.')
    abstract = llm.send_message_for_code(prompt_funcs.abstract_prompt(tags_by_llm, input_info))
    print(f'Abstract generated in {time.time() - starting_time} seconds.')
    performance_dict['abstract'] = time.time() - starting_time
    starting_time = time.time()
    print(f'Generating possible endings.')
    endings_by_llm = llm.send_message_for_format(prompt_funcs.possible_ending_prompt(abstract), PossibleEndings ,model='gpt-4o')
    print(f'Possible endings generated in {time.time() - starting_time} seconds.')
    performance_dict['endings'] = time.time() - starting_time
    starting_time = time.time()
    print(f'Generating dialogue tree.')
    tree_by_llm = llm.send_message_for_format(prompt_funcs.dialogue_prompt(abstract, endings_by_llm, tags_by_llm), format=Tree,model='gpt-4o')
    print(f'Dialogue tree generated in {time.time() - starting_time} seconds.')
    performance_dict['tree'] = time.time() - starting_time
    starting_time = time.time()
    print(f'Total time taken : {time.time() - t_0} seconds.')
    print(f'Performance dict : {performance_dict}')
    #formatted_endings_json = json.loads(endings_by_llm)
    return tree_by_llm, endings_by_llm


In [ ]:
#tree, endings = get_dialogue_json(INPUT_INFO)

Generating tags.
Tags generated in 2.5117878913879395 seconds.
Generating abstract.
Abstract generated in 2.788331985473633 seconds.
Generating possible endings.
Possible endings generated in 23.468002796173096 seconds.
Generating dialogue tree.
Dialogue tree generated in 82.8299708366394 seconds.
Total time taken : 111.59956097602844 seconds.
Performance dict : {'tags': 2.511867046356201, 'abstract': 2.7885429859161377, 'endings': 23.468127965927124, 'tree': 82.83101892471313}


In [10]:
import requests, json

In [11]:
url = 'http://127.0.0.1:5000'

In [17]:
url = "http://127.0.0.1:5000//generate_tags"

payload = json.dumps({
  "input_info": "NPC personality : lazy and shy . NPC mood : confused and amused. Convesation features : dancing, great move, tripping"
})
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

In [21]:
tags = response.json()['tags']

In [23]:
url = "http://127.0.0.1:5000/generate_abstract"
payload = json.dumps({
    "input_info" : INPUT_INFO,
    "tags" : tags
})

response = requests.request("POST", url, headers=headers, data=payload)

In [24]:
abstract = response.json()['abstract']

In [29]:
url = "http://127.0.0.1:5000/generate_possible_endings"
payload = payload = json.dumps({
    "input_info" : INPUT_INFO,
    "tags" : tags,
    "abstract" : abstract
})
response = requests.request("POST", url, headers=headers, data=payload)

In [30]:
endings = response.json()['endings']

In [31]:
url = "http://127.0.0.1:5000/generate_dialogue_tree"
payload = payload = json.dumps({
    "input_info" : INPUT_INFO,
    "tags" : tags,
    "abstract" : abstract,
    "endings" : endings
})
response = requests.request("POST", url, headers=headers, data=payload)

In [32]:
response.json()['tree']


'{"nodes":[{"id":"node1","npc_dialogue":"Hey there! Ever seen a spirit trip while trying to cha-cha-cha? It\'s quite the splash!","tag_of_node":"greeting_funny","player_dialogue_choices":[{"player_dialogue_choice":"Is that why you\'re soaked? Maybe you\'d excel at synchronized swimming!","next_node":"node2"},{"player_dialogue_choice":"Did the fountain deserve an apology, or was it your graceful way of saying hello?","next_node":"node3"}] ,"end_node_effect":{ "positive":{ "parameter_tag":"mood", "value":"0.5" },"neutral":{ "parameter_tag":"restlessness", "value":"0.2" },"negative":{ "parameter_tag":"statusEffect", "value":"\\"confused\\", \\"add\\" : true" }}},{"id":"node2","npc_dialogue":"Ha! You might be onto something. But I\'d rather be floating on clouds than sinking in fountains.","tag_of_node":"amused_friendly","player_dialogue_choices":[{"player_dialogue_choice":"Maybe the clouds are just in need of a good cha-cha too!","next_node":"node4"},{"player_dialogue_choice":"Time for a 